<a href="https://colab.research.google.com/github/jlgrons/CELEHS-DSinA/blob/main/Week_2_Materials/Introduction_to_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Convolutional Neural Networks (CNN)

This notebook guides you through an introduction to CNNs by building CNNs from scratch with the PyTorch packages using the MedMNIST dataset.

The MedMNIST dataset is licensed under Creative Commons Attribution 4.0 International (CC BY 4.0). The dataset is from the following paper:

* Jiancheng Yang, Rui Shi, Donglai Wei, Zequan Liu, Lin Zhao, Bilian Ke, Hanspeter Pfister, Bingbing Ni. "MedMNIST v2: A Large-Scale Lightweight Benchmark for 2D and 3D Biomedical Image Classification". arXiv preprint arXiv:2110.14795, 2021.
* Jiancheng Yang, Rui Shi, Bingbing Ni. "MedMNIST Classification Decathlon: A Lightweight AutoML Benchmark for Medical Image Analysis". IEEE 18th International Symposium on Biomedical Imaging (ISBI), 2021.

---
## Initialization
Below, we provide the code for setting up the environment.  The first section is installing the medmnist dataset in terminal.  The second section initializes some utlitiy packages like tqdm and numpy.  The last two groups are for PyTorch and MedMNIST.

In [ ]:
# medmnist installation
# Terminal command for installing packages. pip is commonly used to install any python package.  Here, we pass in medmnist as the package name.  You can install any existing python packages using the command format pip install XXX, where XXX is the name of the desired package.
!pip install medmnist

# utlity packages
from tqdm import tqdm
import numpy as np

# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

# medmnist python
import medmnist
from medmnist import INFO, Evaluator


The following is the version of MedMNIST and a link to its github page.

In [ ]:
# This is a special print statement using print format.  We are outputting the version of our medmnist dataset and linking the website for the repository and data.
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.1.0 @ https://github.com/MedMNIST/MedMNIST/


---
## Preprocessing and Parameters
We will be working with the PathMNIST dataset in this notebook.  This dataset involves colon pathology and was used for a study involving predicting survival from colorectal cancer histology slides.  It is here where we establish some paramters used during the model training as well as preprecessing for the data.

In [ ]:
# There are a lot of different dataset.  The data_flag field is that specifies which dataset we are going to be pulling from.  The other parameters are self explanatory.
data_flag = 'pathmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
# Learning Rate
lr = 0.001


info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

# Data retrieval into a class object
DataClass = getattr(medmnist, info['python_class'])

In [ ]:
# This transforms our data through tensor building and normalization.
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# In these lines we split the dataset into training and testing data for future use.  These are all objects in the form of a python class.
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

# Dataloader is simply the avenue that allows us to read in the data.  
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)


  0%|          | 0/205615438 [00:00<?, ?it/s]

Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz


---
## Overview of Data
Printing the dataset, we can see some important information in the dataset such as the labels and their corresponding meanings, the description of the dataset, the number of channels in each image, and the task that the dataset is used for.  Following this, we print a montage of the images to gain a general understanding of what some of the images actually look like.

In [ ]:
print(train_dataset)
print("===================")
print(test_dataset)


Dataset PathMNIST (pathmnist)
    Number of datapoints: 89996
    Root location: /root/.medmnist
    Split: train
    Task: multi-class
    Number of channels: 3
    Meaning of labels: {'0': 'adipose', '1': 'background', '2': 'debris', '3': 'lymphocytes', '4': 'mucus', '5': 'smooth muscle', '6': 'normal colon mucosa', '7': 'cancer-associated stroma', '8': 'colorectal adenocarcinoma epithelium'}
    Number of samples: {'train': 89996, 'val': 10004, 'test': 7180}
    Description: The PathMNIST is based on a prior study for predicting survival from colorectal cancer histology slides, providing a dataset (NCT-CRC-HE-100K) of 100,000 non-overlapping image patches from hematoxylin & eosin stained histological images, and a test dataset (CRC-VAL-HE-7K) of 7,180 image patches from a different clinical center. The dataset is comprised of 9 types of tissues, resulting in a multi-class classification task. We resize the source images of 3×224×224 into 3×28×28, and split NCT-CRC-HE-100K into train

In [ ]:
# The .montage() function takes in a parameter length which allows you to set the number of images per row and per column to output in a square montage.  
# This function is specific to the DataClass class.  In this example, we have set the parameter length to 20, and we can count 20 images as the side length of the square montage.
train_dataset.montage(length=20)


NameError: ignored

---
## Creating the CNN
In this module, we create of own CNN class object.  It will consist of 5 layers.  We also deinfe our loss funciton and optimizer based on the task at hand.

In [ ]:
# This is the python class for our convolutional NN.  

class Net(nn.Module):
    def __init__(self, in_channels, num_classes):
        # the super function tells us what class it inherits from.  
        super(Net, self).__init__()

        # Every single one of these layers followed a similar format where we use the overarching Sequential function.  The Conv2D function specifies the type of layer that it is 
        # and established in the number of input channels and kernel size.  BatchNorm2D is a helper function to transform our data into a normalized 2D format.  Relu is the activation 
        # function for our layer to output to the next.  There is also MaxPool2d, which is a funciton that takes in a certain frame of the data and summarizes it by take the maximum value
        # within the kernel size.   Linear is also an activation function.
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Sequential(
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes))
    # The forward function takes in our data and outputs the transformed the data into the classification that we want by running the data through all of the layers
    # that we have created.  The last layer transforms the data into the correct format that we want for classification/labeling.
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Instantiating an instance of the class that we just created.  We pass in the number of channels and the number of classes for the model.
model = Net(in_channels=n_channels, num_classes=n_classes)
    
# These are the loss function and optimizers that we want to deal with for this problem.  
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()
    
# SGD is a stochastic gradient descent function which performs optimization for our function using the model paramters, and lr as the main function we want to optimize.  
# The momentum parameter is a setting that we can use to perform the gradient descent.  This is less important and more on the technical side.
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)



---
## Training and Testing the Model


In [ ]:
# We train the model using a set number of epochs

for epoch in range(NUM_EPOCHS):
    # Here, we tally up the correct and total instances.
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    
    model.train()
    for inputs, targets in tqdm(train_loader):
        # Forward training through our model, adjusting for the loss, and performing backward optimization
        optimizer.zero_grad()
        outputs = model(inputs)
        
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            loss = criterion(outputs, targets)
        else:
            targets = targets.squeeze().long()
            loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()

100%|██████████| 704/704 [02:56<00:00,  3.98it/s]


In [ ]:
# Evaluating the model on the testing dataset

def test(split):
    # Evaluating the model
    model.eval()
    y_true = torch.tensor([])
    y_score = torch.tensor([])
    
    data_loader = train_loader_at_eval if split == 'train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
          # Forward training through our model, adjusting for the loss, and performing backward optimization
            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                outputs = outputs.softmax(dim=-1)
            else:
                targets = targets.squeeze().long()
                outputs = outputs.softmax(dim=-1)
                targets = targets.float().resize_(len(targets), 1)

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)

        # Transformation of data into the correct format
        y_true = y_true.numpy()
        y_score = y_score.detach().numpy()
        
        # Obtaining the performance metrics of AUC and accuracy
        evaluator = Evaluator(data_flag, split)
        metrics = evaluator.evaluate(y_score)
    
        print('%s  auc: %.3f  acc:%.3f' % (split, *metrics))

# Calling the testing function on our traing and test datasets
print('==> Evaluating ...')
test('train')
test('test')

==> Evaluating ...


---
## Additional Resources

The following module provides a different dataset in the MedMNIST corpus.
If you want a more detailed explainer on the compoents in a CNN, you can check the link below.

In [ ]:
data_flag = 'organmnist3d'
download = True

info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])

# load the data
train_dataset = DataClass(split='train',  download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)


In [ ]:
https://poloclub.github.io/cnn-explainer/